In [1]:
from datetime import datetime, timedelta
import pandas as pd

In [2]:
# Creating an example DataFrame
data = {
    'transaction_date': ['2024-03-26T10:30:00', '2024-03-26T10:30:00', '2024-03-26T10:30:00', '2024-03-26T10:30:00', '2024-03-28T10:30:00'],
    'transaction_id': ['TX123', 'TX124', 'TX125', 'TX126', 'TX127'],
    'transaction_message': ['Payment for services', 'Invoice payment', 'Transfer for investment', 'Salary payment', 'Purchase of goods'],
    'transaction_currency': ['USD', 'EUR', 'GBP', 'CAD', 'AUD'],
    'transaction_amount': [1000, 550, 725, 1200, 50],
    'transaction_type': ['MT103', 'pain.001.001.09', '3600', 'MT103', 'pain.001.001.09'],
    'transaction_direction': ['i', 'o', 'io', 'i', 'o'],
    'transaction_status': ['accepted', 'rejected', 'accepted', 'accepted', 'accepted'],
    'instrument_type': ['wire', 'ach/lcy_transfers', 'cash', 'wire', 'ach/lcy_transfers'],
    'originator_full_name': ['John Doe', 'Alice Smith', 'Robert Johnson', 'Michael Williams', 'Elizabeth Brown'],
    'originator_first_name': ['John', 'Alice', 'Robert', 'Michael', 'Elizabeth'],
    'originator_middle_names_patronymic': ['', 'Anne', '', 'Thomas', ''],
    'originator_last_name': ['Doe', 'Smith', 'Johnson', 'Williams', 'Brown'],
    'originator_address': ['123 Main St', '456 Elm St', '789 Oak St', '210 Pine St', '321 Cedar St'],
    'originator_country': ['Iran', 'Vietnam', 'Cameroon', 'Canada', 'Australia'],
    'originator_account_number': ['1234567890', '1234567890', '1234567890', '1234567890', '2468135790'],
    'originator_branch_id': ['NYC123', '', 'LAX456', '', 'TOR789'],
    'originator_bic': ['ABCDEF12XXX', 'GHIJKL34XXX', 'MNOPQR56XXX', 'STUVWX78XXX', 'YZABCD90XXX'],
    'originator_fi_name': ['Bank of America', 'Barclays Bank', 'TD Canada Trust', 'Royal Bank of Canada', 'Westpac Banking Corporation'],
    'originator_fi_country': ['Iran', 'Montserrat', 'CA', 'Canada', 'Australia'],
    'incoming_intermediary_fi_bic': ['XYZ7890XXX;UVW4567XXX', '', '', 'UVW4567XXX', ''],
    'outgoing_intermediary_fi_bic': ['', '', '', '', ''],
    'beneficiary_full_name': ['Jane Doe', 'Bob Johnson', 'Emily Brown', 'David Jones', 'Sarah Taylor'],
    'beneficiary_first_name': ['Jane', 'Bob', 'Emily', 'David', 'Sarah'],
    'beneficiary_middle_names_patronymic': ['', '', '', '', ''],
    'beneficiary_last_name': ['Doe', 'Johnson', 'Brown', 'Jones', 'Taylor'],
    'beneficiary_address': ['321 Elm St', '654 Oak St', '987 Maple St', '147 Pine St', '258 Cedar St'],
    'beneficiary_country': ['Malaysia', 'Montserrat', 'Spain', 'Canada', 'Australia'],
    'beneficiary_account_number': ['9876543210', '2468135790', '2468135790', '6543219870', '8529637410'],
    'beneficiary_branch_id': ['', 'TOR789', '', 'VAN147', ''],
    'beneficiary_bic': ['STUVWX78XXX', 'YZABCD90XXX', 'EFGHIJ12XXX', 'UVW4567XXX', 'QRSTUV23XXX'],
    'beneficiary_fi_name': ['Chase Bank', 'Scotiabank', 'Wells Fargo', 'Bank of Montreal', 'Commonwealth Bank of Australia'],
    'beneficiary_fi_country': ['US', 'CA', 'US', 'Canada', 'Australia']
}

df = pd.DataFrame(data)

In [3]:
# Function to calculate risk factor for round amount payments
def calculate_round_amount_risk(row):
    # Check if transaction amount is a multiple of 25
    if row['transaction_amount'] % 100 == 0:
        return 1  # Round amount payment, assign risk factor 1
    elif row['transaction_amount'] % 50 == 0:
        return 0.5  # Multiple of 25, assign risk factor 0.25
    # Check if transaction amount is a multiple of 50
    elif row['transaction_amount'] % 25 == 0:
        return 0.25  # Multiple of 50, assign risk factor 0.5
    else:
        return 0  # Not a round amount payment, assign risk factor 0

# Apply the function to each row and create a new column 'round_amount_risk'
df['risk_rate'] = df.apply(calculate_round_amount_risk, axis=1)

In [4]:
blacklisted = ["Democratic People's Republic of Korea", "Iran", "Myanmar"]

greylisted = ["Bulgaria", "Burkina Faso", "Cameroon", "Croatia", "Democratic Republic of Congo",
        "Haiti", "Jamaica", "Kenya", "Mali", "Mozambique", "Namibia", "Nigeria",
        "Philippines", "Senegal", "South Africa", "South Sudan", "Syria", "Tanzania",
        "Türkiye", "Vietnam", "Yemen"]

data = {
    "United States": 67, "Switzerland": 70, "Singapore": 67, "Hong Kong": 65, "Luxembourg": 55,
    "Japan": 63, "Germany": 57, "United Arab Emirates": 79, "British Virgin Islands": 71,
    "Guernsey": 71, "China": 66, "Netherlands": 65, "United Kingdom": 47, "Cayman Islands": 73,
    "Cyprus": 62, "South Korea": 64, "Taiwan": 60, "Panama": 73, "Jersey": 63, "Qatar": 74,
    "Italy": 55, "Bahamas": 75, "Thailand": 70, "Vietnam": 81, "Saudi Arabia": 69,
    "Belgium": 53, "Ireland": 47, "Canada": 51, "Spain": 57, "France": 48, "Macao": 63,
    "Israel": 59, "Angola": 79, "Algeria": 79, "Kuwait": 75, "India": 55, "Australia": 56,
    "Malta": 54, "Malaysia": 66, "Liberia": 73, "Kenya": 67, "Nigeria": 65, "Russia": 60,
    "Austria": 55, "Guatemala": 75, "South Africa": 60, "Oman": 74, "Norway": 53,
    "Bermuda": 70, "Sri Lanka": 76, "Marshall Islands": 71, "Bangladesh": 75, "New Zealand": 63,
    "Liechtenstein": 72, "Mauritius": 70, "Egypt": 68, "Portugal": 57, "Anguilla": 75,
    "Turkey": 61, "Bahrain": 68, "Isle of Man": 65, "Romania": 59, "Barbados": 74, "Puerto Rico": 78,
    "Jordan": 72, "Indonesia": 56, "Sweden": 45, "St. Kitts and Nevis": 77, "Venezuela": 72,
    "Ghana": 53, "Uruguay": 58, "Philippines": 67, "Chile": 60, "Pakistan": 66, "Aruba": 71,
    "Hungary": 55, "Lebanon": 65, "Kazakhstan": 63, "Morocco": 66, "Denmark": 49, "Cameroon": 70,
    "Mexico": 53, "Brazil": 49, "Dominican Republic": 65, "Ukraine": 59, "Poland": 46,
    "US Virgin Islands": 72, "Finland": 52, "Seychelles": 72, "Curacao": 76, "Maldives": 75,
    "Czechia": 50, "Tanzania": 69, "Namibia": 71, "Latvia": 55, "Gibraltar": 67,
    "El Salvador": 61, "Rwanda": 72, "Greece": 53, "Croatia": 53, "Slovakia": 53, "Tunisia": 60,
    "Lithuania": 51, "Samoa": 73, "Costa Rica": 56, "Bulgaria": 53, "Peru": 54, "Colombia": 54,
    "Bolivia": 79, "Serbia": 54, "Argentina": 49, "Vanuatu": 76, "Botswana": 57, "Andorra": 55,
    "Belize": 75, "Ecuador": 52, "Paraguay": 66, "Monaco": 74, "Montenegro": 61,
    "Turks and Caicos Islands": 76, "Fiji": 70, "St. Vincent and the Grenadines": 67,
    "Albania": 54, "North Macedonia": 62, "Estonia": 44, "Iceland": 42, "Antigua and Barbuda": 77,
    "Dominica": 65, "Kosovo": 69, "Trinidad and Tobago": 69, "Cook Islands": 70, "Grenada": 66,
    "St. Lucia": 72, "Guam": 70, "American Samoa": 69, "Brunei": 73, "Slovenia": 36,
    "Gambia": 73, "Nauru": 59, "San Marino": 60, "Montserrat": 74,
}

# Create a dictionary using the data
secrecy_scores = {country: score for country, score in data.items()}

# Function to calculate risk rate for each row
def calculate_risk_rate(row):
    originator_country = row['originator_country']
    beneficiary_country = row['beneficiary_country']
    
    # Initialize points
    originator_points = 0

    # Check if originator country is blacklisted or greylisted and calculate points based on their scores
    if originator_country in blacklisted:
        originator_points = 1 * (secrecy_scores.get(beneficiary_country, 0) / 100)
    elif originator_country in greylisted:
        originator_points = 0.66 * (secrecy_scores.get(beneficiary_country, 0) / 100)
    else:
        originator_points = 0.33 * (secrecy_scores.get(beneficiary_country, 0) / 100)
        
    country_risk_rate = originator_points

    # Add round amount risk to the existing risk_rate
    total_risk_rate = row['risk_rate'] + country_risk_rate

    return total_risk_rate

# Apply the function to each row and update the 'risk_rate' column with the combined risk rate
df['risk_rate'] = df.apply(calculate_risk_rate, axis=1)

In [5]:
# Function to calculate risk score for each transaction
def calculate_risk_score(row):
    # Initialize risk score
    risk_score = 0
    
    # Filter DataFrame to include transactions within 24 hours of the current transaction
    start_time = datetime.fromisoformat(row['transaction_date']) - timedelta(days=1)
    end_time = datetime.fromisoformat(row['transaction_date'])
    
    recent_transactions = df[(df['transaction_date'] >= start_time.isoformat()) & (df['transaction_date'] <= end_time.isoformat())]
    
    # Originator frequency and beneficiary frequency within 24 hours
    originator_frequency = recent_transactions[recent_transactions['originator_account_number'] == row['originator_account_number']].shape[0]
    beneficiary_frequency = recent_transactions[recent_transactions['beneficiary_account_number'] == row['beneficiary_account_number']].shape[0]
    
    # Check if originator or beneficiary has more than 3 transactions within 24 hours
    if originator_frequency > 3 or beneficiary_frequency > 3:
        risk_score += 0.4
    
    # Check transaction amount
    if row['transaction_amount'] < 100:
        risk_score += 0.3
    
    # Check for geographical spread
    unique_originator_countries = recent_transactions[recent_transactions['originator_account_number'] == row['originator_account_number']]['originator_country'].nunique()
    unique_beneficiary_countries = recent_transactions[recent_transactions['beneficiary_account_number'] == row['beneficiary_account_number']]['beneficiary_country'].nunique()
    
    if unique_originator_countries > 1 or unique_beneficiary_countries > 1:
        risk_score += 0.3
    
        # Add round amount risk to the existing risk_rate
    risk_score = row['risk_rate'] + risk_score

    return risk_score

# Apply the function to each row and create a new column 'risk_score'
df['risk_rate'] = df.apply(calculate_risk_score, axis=1)

In [6]:
# import random

# def generate_nested_transactions(num_transactions, max_children=2):
#     nested_transactions = {}
#     all_transaction_ids = [f'TX{i}' for i in range(1, num_transactions + 1)]

#     for transaction_id in all_transaction_ids:
#         # Randomly choose the number of children for each transaction
#         num_children = random.randint(0, max_children)
#         nested_transactions[transaction_id] = random.sample(all_transaction_ids, num_children)

#     return nested_transactions

# # Example usage:
# nested_transactions = generate_nested_transactions(num_transactions=5, max_children=2)

# # Function to calculate nesting level for a transaction
# def calculate_nesting_level(transaction_id, parent_id=None, depth=0):
#     # Base case: If the transaction_id is not found, return the depth
#     if transaction_id not in nested_transactions:
#         return depth
    
#     # Recursively calculate the nesting level for nested transactions
#     nested_depths = [calculate_nesting_level(nested_id, transaction_id, depth + 1) for nested_id in nested_transactions[transaction_id]]
    
#     # Return the maximum nesting depth among the nested transactions
#     return max(nested_depths)

# # Apply the function to each row and create a new column 'nesting_level'
# df['nesting_level'] = df['transaction_id'].apply(calculate_nesting_level)

# # Calculate risk rate based on nesting level
# def calculate_risk_rate_based_on_nesting(nesting_level):
#     if nesting_level == 0:
#         total_risk_rate = row['risk_rate'] + country_risk_rate
#         return total_risk_rate
#     elif nesting_level == 1:
#         return 0.2
#     elif nesting_level == 2:
#         return 0.5
#     else:
#         return 1.0  # High risk for higher nesting levels

# # Apply the function to each row and create a new column 'risk_rate_nesting'
# df['risk_rate_nesting'] = df['nesting_level'].apply(calculate_risk_rate_based_on_nesting)

In [8]:
df['risk_rate'].head()

0    2.3600
1    1.6884
2    1.3262
3    1.8683
4    0.9848
Name: risk_rate, dtype: float64